In [0]:
%sql
-- Create catalog
CREATE CATALOG IF NOT EXISTS ecommerce_catalog;
-- Create schemas
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.bronze;
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.silver;
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.gold;

In [0]:
%sql
SHOW CATALOGS;
SHOW SCHEMAS IN ecommerce_catalog;

databaseName
bronze
default
gold
information_schema
silver


In [0]:
%sql
SHOW VOLUMES IN ecommerce_catalog.bronze;

database,volume_name
bronze,bronze_volume


In [0]:
%sql
DESCRIBE VOLUME ecommerce_catalog.bronze.bronze_volume;

name,catalog,database,owner,storage_location,volume_type,comment,securable_type,securable_kind
bronze_volume,ecommerce_catalog,bronze,ganapathisking@gmail.com,,MANAGED,null,VOLUME,VOLUME_DB_STORAGE


In [0]:
display(spark.sql("SHOW VOLUMES IN ecommerce_catalog.bronze"))
display(spark.sql("SHOW VOLUMES IN ecommerce_catalog.silver"))
display(spark.sql("SHOW VOLUMES IN ecommerce_catalog.gold"))

database,volume_name
bronze,bronze_volume


database,volume_name
silver,silver_volume


database,volume_name
gold,gold_volume


In [0]:
# Create volumes under your Unity Catalog & Schema
catalog = "ecommerce_catalog"
schema = "default"
volumes = ["bronze.bronze_volume", "silver.silver_volume", "gold.gold_volume"]
for volume in volumes:
    try:
        display(spark.sql("CREATE VOLUME IF NOT EXISTS ecommerce_catalog.bronze.bronze_volume"))
        display(spark.sql("CREATE VOLUME IF NOT EXISTS ecommerce_catalog.silver.silver_volume"))
        display(spark.sql("CREATE VOLUME IF NOT EXISTS ecommerce_catalog.gold.gold_volume"))
        print(f"Volume '{catalog}.{schema}.{volume}' created successfully.")
    except Exception as e:
        if "VOLUME_ALREADY_EXISTS" in str(e):
            print(f"Volume '{catalog}.{schema}.{volume}' already exists. ")
        else:
            print(f"❌ Failed to create volume '{catalog}.{schema}.{volume}':")
            print(str(e))

Volume 'ecommerce_catalog.default.bronze.bronze_volume' created successfully.


Volume 'ecommerce_catalog.default.silver.silver_volume' created successfully.


Volume 'ecommerce_catalog.default.gold.gold_volume' created successfully.


In [0]:
try:
  # Read existing Delta data
  bronze_df = spark.read.format("delta").load("dbfs:/Volumes/workspace/ecommerce/bronze/events_delta")
  silver_df = spark.read.format("delta").load("dbfs:/Volumes/workspace/ecommerce/silver/events_delta")
  gold_df   = spark.read.format("delta").load("dbfs:/Volumes/workspace/ecommerce/gold/revenue_by_brand")
  # Write Delta tables into volumes inside the UC
  bronze_df.write.format("delta").mode("overwrite").save("/Volumes/ecommerce_catalog/bronze/bronze_volume/events_delta")
  silver_df.write.format("delta").mode("overwrite").save("/Volumes/ecommerce_catalog/silver/silver_volume/events_delta")
  gold_df.write.format("delta").mode("overwrite").save("/Volumes/ecommerce_catalog/gold/gold_volume/revenue_by_brand")
except Exception as e:
  print(str(e))

In [0]:
bronze_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_catalog.bronze.events")  # Register Bronze as managed table
silver_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_catalog.silver.events_cleaned")  # Register Silver as managed table
gold_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_catalog.gold.revenue_by_brand")  # Register Gold as managed table

In [0]:
%sql
GRANT ALL PRIVILEGES ON CATALOG ecommerce_catalog TO `account users`;  -- Give yourself full control on the catalog (this usually works)

In [0]:
%sql
CREATE OR REPLACE VIEW ecommerce_catalog.gold.revenue_by_brand_public AS
SELECT brand, total_revenue
FROM ecommerce_catalog.gold.revenue_by_brand;

In [0]:
%sql
REVOKE SELECT ON TABLE ecommerce_catalog.gold.revenue_by_brand FROM `account users`;
GRANT SELECT ON VIEW ecommerce_catalog.gold.revenue_by_brand_public TO `account users`;